SMT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SMT version.

Necessary libraries:

In [ ]:
!pip install z3-solver

Necessary imports:

In [1]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [40]:
# open the file in Instances folder
f = open("inst05.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 2
items: 3
load_size: [18, 30]
item_size: [20, 17, 6]
distance:
 [[ 0 21 86 99]
 [21  0 71 80]
 [92 71  0 61]
 [59 80 61  0]]


Other variables:

In [41]:
max_path_length = n-(m-1)
paths = [[Int("p_%s_%s" % (i,j)) for j in range(max_path_length)] for i in range(m)] 
# Create a solver instance
s = Solver()


Main constraints:

In [57]:
constraints = []
constraints.append(n>=m)

# Each item has a size > 0 and <= max(load size)
for i in range(n):
    constraints.append(And(item_size[i] > 0, item_size[i] <= max(load_size)))

# Each item is delivered at most by one courier, and at least by one courier
for i in range(n):
    constraints.append(Sum([If(paths[c][j] == i, 1, 0) for c in range(m) for j in range(max_path_length)]) == 1)

max_distance = Int("max_distance")
courier_loads_sum = [Int("courier_loads_sum_%s" % c) for c in range(m)] 
constraints.append(max_distance >= 0)
for c in range(m):
    constraints.append(courier_loads_sum[c] > 0)
s.add(constraints)
courier_distances = [[10000] for c in range(m)]
courier_loads = [[0] for c in range(m)]
for l in range(1000):
    if s.check() == sat:
        model = s.model()
        # get the values of the paths
        paths_values = [[model[paths[i][j]] for j in range(max_path_length)] for i in range(m)]
        # get path for each courier as a list of items
        paths_items = [[paths_values[i][j].as_long() for j in range(max_path_length) if paths_values[i][j].as_long() != n] for i in range(m)]
        # get the total distance for each courier by adding also the distance from the depot to the first item and from the last item to the depot
        for c in range(m):
            dist = distance[n][paths_items[c][0]] + distance[paths_items[c][-1]][n]
            for i in range(len(paths_items[c])-1):
                dist += distance[paths_items[c][i]][paths_items[c][i+1]]
            courier_distances[c][0] = dist
            courier_loads[c][0] = sum([item_size[i] for i in paths_items[c]])
        
        s.add(max_distance < max([courier_distances[c][0] for c in range(m)]))
        s.add(And([courier_loads_sum[c] < courier_loads[c][0] for c in range(m)]))
        
        
print("courier_distances:", courier_distances)
print("courier_loads:", courier_loads)
  



courier_distances: [[122], [160]]
courier_loads: [[6], [37]]
